In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
train= pd.read_table('../input/train.tsv',engine= 'c')
test= pd.read_table('../input/test.tsv',engine= 'c')
tr_rows = train.shape[0]

In [ ]:
df = pd.concat([train,test],axis = 0)

In [ ]:
df.shape,train.shape, test.shape, test.shape[0]/train.shape[0]

In [ ]:
print("Handling missing values...")
def handle_missing(dataset):
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

df = handle_missing(df)

print(df.shape)


In [ ]:
#PROCESS CATEGORICAL DATA
'''print("Handling categorical variables...")
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(np.hstack([train.category_name, test.category_name]))
train.category_name = le.transform(train.category_name)
test.category_name = le.transform(test.category_name)

le.fit(np.hstack([train.brand_name, test.brand_name]))
train.brand_name = le.transform(train.brand_name)
test.brand_name = le.transform(test.brand_name)
del le'''

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df= 10)
cv.fit(np.hstack([df.name.str.lower()]))
X_name_df = cv.transform(df.name)


In [ ]:
X_name_df = X_name_df.tocsr()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features=10000,
                     ngram_range=(1, 3),
                     stop_words='english')
df_item_description = tv.fit_transform(df['item_description'])
print("tfidf done")

In [ ]:
df.head()

In [ ]:
from scipy.sparse import csr_matrix, hstack
y = hstack((X_name_df,df_item_description)).tocsr()


In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer(sparse_output= True)
lb.fit(df.brand_name)
brand_df = lb.transform(df.brand_name)           

In [ ]:
df_dummies = csr_matrix(pd.get_dummies(df[['item_condition_id', 'shipping']],
                                          sparse=True).values)


In [ ]:
'''category split'''

dfcat = df['category_name'].str.split('/', expand=True)
dfcat.columns = ['cat{}'.format(x+1) for x in dfcat.columns]

dfcat= dfcat.drop(["cat4","cat5"],axis=1)



In [ ]:
def handle_missingcat(dataset):
    dataset.cat1.fillna(value="missing", inplace=True)
    dataset.cat2.fillna(value="missing", inplace=True)
    dataset.cat3.fillna(value="missing", inplace=True)
    return (dataset)

dfcat = handle_missingcat(dfcat)

cv = CountVectorizer()
df_category1 = cv.fit_transform(dfcat['cat1'])
df_category2 = cv.fit_transform(dfcat['cat2'])
df_category3 = cv.fit_transform(dfcat['cat3'])


In [ ]:
df_sparse_merge = hstack((df_dummies, y, brand_df, df_category1, df_category2, df_category3)).tocsr()
print('data processed')

In [ ]:
y_df = np.log1p(df["price"])
train_df = df_sparse_merge[:tr_rows,:]
test_df = df_sparse_merge[tr_rows:,:]
y_train = y_df[:tr_rows]
y_test = df['price'][tr_rows:].values
from sklearn.model_selection import train_test_split
import lightgbm as lgb 
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
print('splitting done')

In [ ]:
train1 = train_df
#train2 = train_df[900000:]
target1 = y_train
#target2 = y_train[900000:]
train_X, valid_X, train_y, valid_y = train_test_split(train1, target1, test_size = 0.15, random_state = 144) 
d_train = lgb.Dataset(train_X, label=train_y, max_bin=8192)
d_valid = lgb.Dataset(valid_X, label=valid_y, max_bin=8192)
watchlist = [d_train, d_valid]
params = {
    'learning_rate': 0.65,
    'application': 'regression',
    'max_depth': 3,
    'num_leaves': 60,
    'verbosity': -1,
    'metric': 'RMSE',
    'data_random_seed': 1,
    'bagging_fraction': 0.5,
    'nthread': 4
}
model = lgb.train(params, train_set=d_train, num_boost_round=4000, valid_sets=watchlist, \
early_stopping_rounds=1000, verbose_eval=1000) 
predsL = model.predict(train1)#result 0.7
tpredsL = model.predict(test_df)
print("lgb done")
#ridge

model2 = Ridge(alpha=.9, copy_X=True, fit_intercept=True, max_iter=100,normalize=False, random_state=101, solver='auto', tol=0.01)
model2.fit(train1, target1)
predsR = model2.predict(train1)
tpredsR = model2.predict(test_df)
print("R1 done")
from xgboost import XGBRegressor
model3 = XGBRegressor(n_jobs = -1)
model3.fit(train1,target1)
predsX = model3.predict(train1)
tpredsX = model3.predict(test_df)
print("xgb done")
model4 = Ridge(solver="sag", fit_intercept=True, random_state=205)
model4.fit(train1,target1)
predsR2 = model4.predict(train1)
tpredsR2 = model4.predict(test_df)
print("R2 done")
ensem = 0.1*(predsL)
#rmsle(y_test,(y_pred+np.expm1(predsR2))/2)
ensem += 0.4*(predsR)
ensem  += 0.1*(predsX)
ensem  += 0.4*(predsR2)
ensemt = 0.1*(tpredsL)
ensemt += 0.4*(tpredsR)
ensemt  += 0.1*(tpredsX)
ensemt  += 0.4*(tpredsR2)
train_for_next = pd.DataFrame([predsR2,predsR,predsL,predsX,ensem]).transpose()
test_for_next = pd.DataFrame([tpredsR2,tpredsR,tpredsL,tpredsX,ensemt]).transpose()
'''modelR3 = Ridge(alpha=.9, copy_X=True, fit_intercept=True, max_iter=100,normalize=False, random_state=101, solver='auto', tol=0.01)
modelR3.fit(train_for_next, target2)
predsR3 = modelR3.predict(test_for_next)'''



'''regressor = RandomForestRegressor(n_estimators = 100,n_jobs = -1, random_state = 0)
regressor.fit(train_df, y_train)
y_predrf = regressor.predict(test_df)'''


'''dtregressor = DecisionTreeRegressor(max_depth= 50,random_state = 0)
dtregressor.fit(train_df,y_train)
y_preddt = dtregressor.predict(train_df[:10000])
print("dt done")'''


In [ ]:

#y_pred += 0.2*np.expm1(y_predrf)
#y_pred += 0.2*np.expm1(y_preddt)
#rmsle(y_test,y_pred)
#y_pred = np.expm1(predsR3)


In [ ]:
'''result = pd.DataFrame([y_pred,y_test]).transpose()
result.columns = ['pred_ensemble','test']
result.boxplot(figsize = (12,6))'''
#rmsle(np.expm1(predsR3),np.expm1(y_pred_nn))
#print(np.expm1(predsR3),y_pred)


In [ ]:
'''result = pd.DataFrame([np.expm1(predsR2),np.expm1(predsR),np.expm1(predsL),np.expm1(predsX),y_pred,y_test]).transpose()
result.columns = ['ridge2','ridge','lgbm','xgb','pred_ensemble','test']
result.boxplot(figsize = (14,9))'''
%time
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_for_next_scaled = np.zeros(train_for_next.shape)
test_for_next_scaled = np.zeros(test_for_next.shape)
train_for_next_scaled_sin = np.zeros(train_for_next.shape)
test_for_next_scaled_sin = np.zeros(test_for_next.shape)
train_for_next_scaled_cos = np.zeros(train_for_next.shape)
test_for_next_scaled_cos = np.zeros(test_for_next.shape)
for i in range(5):
    train_for_next_scaled[:,i] = sc.fit_transform(train_for_next.iloc[:,i].reshape(-1,1))[:,0]
    test_for_next_scaled[:,i] = sc.fit_transform(test_for_next.iloc[:,i].reshape(-1,1))[:,0]
    train_for_next_scaled_sin[:,i] = np.sin(train_for_next_scaled[:,i])
    test_for_next_scaled_sin[:,i] =  np.sin(test_for_next_scaled[:,i])
    train_for_next_scaled_cos[:,i] = np.cos(train_for_next_scaled[:,i])
    test_for_next_scaled_cos[:,i] =  np.cos(test_for_next_scaled[:,i])
NN_train = np.hstack((np.asmatrix(train_for_next_scaled),np.asmatrix(train_for_next_scaled_sin),np.asmatrix(train_for_next_scaled_cos))) 
NN_test = np.hstack((np.asmatrix(test_for_next_scaled),np.asmatrix(test_for_next_scaled_sin),np.asmatrix(test_for_next_scaled_cos))) 
#from sklearn.neural_network import MLPRegressor 
#nn = MLPRegressor(hidden_layer_sizes=(50,50,50),max_iter=150,verbose= True,random_state= 0,tol=0.0000000001)
#nn.fit(NN_train,target1)
finalxg = XGBRegressor(n_jobs = -1)
finalxg.fit(NN_train,target1)
y_pred_nn = finalxg.predict(NN_test)
y_pred = np.expm1(y_pred_nn)

In [ ]:
#rmsle(y_test,(np.expm1(predsL)+np.expm1(predsX)+np.expm1(predsR)+np.expm1(predsR2)+np.expm1(y_predrf))/5)
 


In [ ]:
submission = pd.read_csv('../input/sample_submission.csv',engine = 'c')
submission['price'] = y_pred
print("submitting...")
submission.to_csv('ensemblerrlxdt.csv',index = False)